<a href="https://colab.research.google.com/github/garys-io/crispy-robot/blob/main/Calgary_communities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# !pip install requests
import requests
from pprint import pprint as p
from bs4 import BeautifulSoup
import pathlib
import os

In [18]:
dataDir = '/content/data'

pdfFiles = [file for file in os.listdir(dataDir) if file.endswith('.pdf')]
print(f'Found {len(pdfFiles)} pdf files at dataDir') # ls -al data | grep .pdf | wc -l

# fetch pdf files if there are less than 197 expected
fetchPdfs = len(pdfFiles) < 197
print('fetchPdfs', fetchPdfs)
print()

txtFiles = [file for file in os.listdir(dataDir) if file.endswith('.txt')]
print(f'Found {len(txtFiles)} txt files at dataDir')

# extract text out of pdf files
extractTexts = len(txtFiles) < 197
print('extractTexts', extractTexts)

Found 197 pdf files at dataDir
fetchPdfs False

Found 0 txt files at dataDir
extractTexts True


In [19]:
if fetchPdfs:
  # Gather data
  calgaryCommuniitesUrl = 'https://www.calgary.ca/csps/cns/research-and-strategy/community-profiles/community-profiles.html'

  r = requests.get(calgaryCommuniitesUrl)
  # p(r.text)

  # parse
  soup = BeautifulSoup(r.text, 'html.parser')
  aTags = soup.find_all('a')
  print(f"Found {len(aTags)} anchor tags")

  communityPageUrls = [t.attrs['href'] for t in aTags
                      if 'href' in t.attrs
                      and t.attrs['href'].startswith('https://www.calgary.ca/csps/cns/social-research-policy-and-resources/community-profiles/')]
  print(f"Found {len(communityPageUrls)} community page URLs")
  # p(aTagsForCommunities)

  # Gather data for individual community by downloading their profile pdf
  for communityPageUrl in communityPageUrls:
    # p(communityPageUrl)
    r = requests.get(communityPageUrl)
    soup = BeautifulSoup(r.text, 'html.parser')
    pdfUrls = [t.attrs['href'] for t in soup.find_all('a')
              if t.attrs['href']
              and t.attrs['href'].startswith('/content/')]
    # print(f"Found {len(pdfUrls)} pdf urls")

    for pdfUrl in pdfUrls:
      url = f"https://www.calgary.ca{pdfUrl}"
      filename = pdfUrl.split('/')[-1]
      with open(f'{dataDir}/{filename}', 'wb') as f:
        r = requests.get(url)
        f.write(r.content)
        print(f"Saved: {filename}")
      break # only first pdf is important

In [72]:
# !pip install PyPDF2
import PyPDF2

pdfPagesParseConfig = [
  {
    'headerCutoff': 730,
    'footerCutoff': 1185
    
  }
]

if extractTexts:
  # inspecting first file
  # for pdfFile in pdfFiles:
    # with open(f'{dataDir}/{pdfFile}', 'rb') as f:
  with open(f'{dataDir}/abbeydale.pdf', 'rb') as f:
    pdfReader = PyPDF2.PdfFileReader(f)
    pageObj = pdfReader.getPage(0).extractText()
    pageText = pageObj[pdfPagesParseConfig[0]['headerCutoff']
                       :pdfPagesParseConfig[0]['footerCutoff']].strip()
    pageLines = [line for line in pageText.split('\n') if line.strip()]

    [
      pop2016,
      _,
      _,
      _,
      _,
      youthPop2016,
      _,
      _,
      _,
      _,
      _,
      _,
      _,
      _,
      _,
      _,
      _,
      _,
      _,
      _,
      _,
      _,
      _,
      _,
      _,
      _,
      _,
      _,
      _,
      _,
      _,
      _,
    ] = pageLines
    print(pop2016, youthPop2016)
    # ] = [
    #   '6,150',
    #   '0',
    #   '-',
    #   '14 years:',
    #   '65+ years:',
    #   '22',
    #   '%',
    #   '8',
    #   '%',
    #   '1,222,390',
    #   '19%',
    #   '10%',
    #   'Per cent ',
    #   'households spending 30% or more of ',
    #   'total income on shelter in 2016',
    #   'Per cent Immigrants in 2016',
    #   '21%',
    #   '33',
    #   '%',
    #   '22%',
    #   '31%',
    #   'Median total household income (before tax) in ',
    #   '2015:',
    #   'Per ',
    #   'cent individuals who speak English most ',
    #   'often at home',
    #   '$',
    #   '81,232',
    #   '72%',
    #   '$$$',
    #   '$97,329',
    #   '75%'
    # ]
    p(pageLines)
    # break


6,150 22
['6,150',
 '0',
 '-',
 '14 years:',
 '65+ years:',
 '22',
 '%',
 '8',
 '%',
 '1,222,390',
 '19%',
 '10%',
 'Per cent ',
 'households spending 30% or more of ',
 'total income on shelter in 2016',
 'Per cent Immigrants in 2016',
 '21%',
 '33',
 '%',
 '22%',
 '31%',
 'Median total household income (before tax) in ',
 '2015:',
 'Per ',
 'cent individuals who speak English most ',
 'often at home',
 '$',
 '81,232',
 '72%',
 '$$$',
 '$97,329',
 '75%']
